In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
#載入數據集，one_hot是把数据转化为只有0和1的形式
mnist = input_data.read_data_sets("MNIST_data", one_hot = True) #這步有時候會失效

#因為數據集很大，故我們要用stochastic gradient descent，
#會將資料集分批次（一次100张）放入神经网络进行训练，
#並不會一次將所有資料拿來train (計算量很大)
#每一個批次的大小
batch_size = 100 

#計算一共有多少批次，训练集数量mnist.train.num_examples 
# // 在python中表示取商
n_batch = mnist.train.num_examples // batch_size      

#定义兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
x = tf.placeholder(tf.float32, [None, 784]) # 28 * 28 = 784，None值变为100
y = tf.placeholder(tf.float32, [None, 10]) #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量
keep_prob = tf.placeholder(tf.float32)
# lr即为learning rate，学习率
lr = tf.Variable(0.001, dtype = tf.float32)  #初始学习率定义的是0.001


#创建一個簡單的神經網路 (输入层784个神经元，输出层總共10個神經元，即十个标签)
#隐藏层1
W1 = tf.Variable(tf.truncated_normal([784, 500],stddev = 0.1))              #权值
    # b = tf.Variable(tf.zeros([1, 10]))                #偏置值
b1 = tf.Variable(tf.zeros([500])+0.1)  
L1 = tf.nn.tanh(tf.matmul(x,W1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob)

#隐藏层2
W2 = tf.Variable(tf.truncated_normal([500, 300],stddev = 0.1))              #权值
b2 = tf.Variable(tf.zeros([300])+0.1)  
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

#隐藏层3
W3 = tf.Variable(tf.truncated_normal([300, 10],stddev = 0.1))              #权值
b3 = tf.Variable(tf.zeros([10])+0.1)  
prediction = tf.nn.softmax(tf.matmul(L2_drop, W3) + b3)  #预测值，用到softmax


#交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction)) #交叉熵

#训练
train_step = tf.train.AdamOptimizer(lr).minimize(loss) 

#初始化变量 
init = tf.global_variables_initializer()

#测试训练的准确率，求准确率的方法
#如果y標籤最大的值，與prediction標籤最大的值相等，則回傳true
#結果存在一個 boolean 的变量correct_prediction中

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
                              
 #argmax 返回一維張量中最大的值所在的位置
 # 求标签y里面最大的值在哪个位置即标签
 # tf.argmax(prediction, 1)预测 概率最大就会判定识别的这张图片是属于哪个标签的
 # (tf.argmax(y, 1)，真实样本的y存放的都是0或1，哪位是1就会返回哪位的值
 #  然后再比较上面两者，是否一样
                             
                              
# 求准确率
# 轉換資料格式 boolean 轉成 32位的float，接著再取平均值，得到准确率
# true转换为1.0，false转换为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         
                              
#開始training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(51): #总共疊代51次（周期） (outer loop)，把所有的图片训练51次
        #每迭代一个周期，都会给学习率重新赋值
        #0.95 ** epoch，指第epoch次迭代就是(0.95^epoch)，然后赋值给lr
        # 学习率逐渐减小（最开始模型是比较混乱的，给它一个大的学习率可以让它快速收敛
        #                 当收敛到全局最小值向其靠近的时候，就逐渐降低学习率，便于靠近）
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        
        #每一次 outer loop 不一次拿所有的數據集，來做 Gradient desent，這就是 stochastic gradient descent
        for batch in range(n_batch):  #每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs图片数据, batch_ys图片标签
            # mnist.train.next_batch(batch_size)是获取下一个一百张图片               
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
                              
              #拿來feed 的 dictionary                  
            sess.run(train_step, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 1.0})
        
#         learnin_rate = sess.run(lr)   #手敲代码拼写单词的时候不小心少输一个字母结果程序报错
        learning_rate = sess.run(lr)
        #testing data feed dictionary
        acc = sess.run(accuracy, feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
                   
        # str(epoch)周期数
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(acc) + ",Learning Rate = " + str(learning_rate))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter =0, Testing Accuracy =0.952,Learning Rate = 0.001
Iter =1, Testing Accuracy =0.9617,Learning Rate = 0.00095
Iter =2, Testing Accuracy =0.9678,Learning Rate = 0.0009025
Iter =3, Testing Accuracy =0.9688,Learning Rate = 0.000857375
Iter =4, Testing Accuracy =0.9705,Learning Rate = 0.00081450626
Iter =5, Testing Accuracy =0.9726,Learning Rate = 0.0007737809
Iter =6, Testing Accuracy =0.9734,Learning Rate = 0.0007350919
Iter =7, Testing Accuracy =0.9771,Learning Rate = 0.0006983373
Iter =8, Testing Accuracy =0.975,Learning Rate = 0.0006634204
Iter =9, Testing Accuracy =0.9784,Learning Rate = 0.0006302494
Iter =10, Testing Accuracy =0.9783,Learning Rate = 0.0005987369
Iter =11, Testing Accuracy =0.9783,Learning Rate = 0.0005688001
Iter =12, Testing Accuracy =0.98,Learning Rate = 0.0005403601
Ite